In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ''

In [49]:
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, get_scheduler, AutoModelForSequenceClassification
from datasets import Dataset
import numpy as np
from evaluate import load as load_metric
import numpy as np
from transformers import pipeline
import pandas as pd
from sklearn.metrics import classification_report
import torch

from tqdm import tqdm
from typing import List, Dict

from flair.data import Sentence
from flair.models import TextClassifier

In [55]:
model_name = "./result"  # Replace with your model's path
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [56]:
# Function to predict labels for a list of strings with batching
def predict(texts: List[str], batch_size: int = 16):
    model.eval()
    
    all_results = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i + batch_size]

        # Tokenize the batch
        inputs = tokenizer(batch_texts, padding="longest", truncation=True, max_length=256, return_tensors="pt")
        
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            probabilities = torch.nn.functional.softmax(logits, dim=-1)
            predictions = torch.argmax(logits, dim=-1)

        for idx in range(predictions.size(0)):
            class_id = predictions[idx].item()
            prob = probabilities[idx][class_id].item()  # Get probability of the predicted class
            all_results.append((class_id, prob))  # Store (class_id, probability))

    return all_results

In [57]:
%%time
result = predict(["Услуга", "Непонятно"] *20000 )

  0%|          | 1/2500 [00:20<14:21:19, 20.68s/it]

KeyboardInterrupt



In [50]:
model = TextClassifier.load("models/USER_bge_m3_1731781657/model_epoch_4.pt")

In [51]:
sentences = [ Sentence(sentence) for sentence in ["Услуга", "Непонятно"] *20000]

In [53]:
model.predict(sentences, mini_batch_size=16, verbose=True)

Batch inference:   0%|          | 4/2500 [01:08<11:50:36, 17.08s/it]
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fb8b18e5780>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 

In [29]:
len([    0,  1231,  7717,     4,   628, 37364,  3926,    32,  1231,  7717,
             4,   628, 37364,  3926,    32,  1231,  7717,     4,   628, 37364,
          3926,    32, 13557,  7717,     4,   628, 37364,  3926,    32, 13557,
          7717,     4,   628, 37364,  3926,    32, 13557,  7717,     4,   628,
         37364,  3926,    32, 13557,  7717,     4,   628, 37364,  3926,    32,
         13557,  7717,     4,   628, 37364,  3926,    32, 13557,  7717,     4,
           628, 37364,  3926,    32, 13557,  7717,     4,   628, 37364,  3926,
            32, 13557,  7717,     4,   628, 37364,  3926,    32, 13557,  7717,
             4,   628, 37364,  3926,    32,     2])

86